In [1]:
import os

In [4]:
%pwd

'c:\\Users\\mehak\\OneDrive\\NLP_Project\\Text-Summarizer---NLP'

In [3]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) 
class ModelTrainerConfig:  
    root_dir: Path
    data_path: Path
    model: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int
    learning_rate: float

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir =  config.root_dir,
            data_path = config.data_path,
            model = config.model,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            learning_rate = params.learning_rate
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\mehak\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-02-04 11:05:12,492: INFO: config: PyTorch version 2.4.1 available.]


In [9]:
from transformers import AdamW

In [ ]:
#device = torch.cuda.is_available()

In [11]:
#device

False

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):

        tokenizer = AutoTokenizer.from_pretrained(self.config.model)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model)

        for param in model_pegasus.parameters():
            param.requires_grad = False

        for param in model_pegasus.model.encoder.layers[-2:].parameters():
            param.requires_grad = True

        for param in model_pegasus.model.decoder.layers[-2:].parameters():
            param.requires_grad = True

        dataset = load_from_disk(self.config.data_path)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps, learning_rate=5e-5
            ) 

        optimizer = AdamW([
            {"params": model_pegasus.model.encoder.layers[-2:].parameters(), "lr": 3e-5},
            {"params": model_pegasus.model.decoder.layers[-2:].parameters(), "lr": 3e-5},
        ], lr=5e-5, weight_decay=self.config.weight_decay)

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            optimizers=(optimizer, None)
        )

        trainer.train()

        # Save the fine-tuned model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-02-04 11:05:32,279: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-04 11:05:32,290: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-04 11:05:32,296: INFO: common: created directory at: artifacts]
[2025-02-04 11:05:32,299: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\mehak\anaconda3\envs\nlp\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\mehak\anaconda3\envs\nlp\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\mehak\AppData\Local\Temp\ipykernel_29668\2742270778.py:35: FutureWarning: `tokenizer`

{'loss': 3.1692, 'grad_norm': 54.798912048339844, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.04}


  3%|▎         | 20/690 [44:10<22:40:37, 121.85s/it]

{'loss': 3.0284, 'grad_norm': 45.83332824707031, 'learning_rate': 1.2000000000000002e-06, 'epoch': 0.09}


...

{'loss': 1.9913, 'grad_norm': 13.256025314331055, 'learning_rate': 1.5789473684210526e-06, 'epoch': 2.95}


100%|██████████| 690/690 [18:32:20<00:00, 63.01s/it]c:\Users\mehak\anaconda3\envs\nlp\lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'loss': 1.9793, 'grad_norm': 14.96561336517334, 'learning_rate': 0.0, 'epoch': 3.0}


100%|██████████| 690/690 [18:32:25<00:00, 96.73s/it]


{'train_runtime': 66745.8018, 'train_samples_per_second': 0.662, 'train_steps_per_second': 0.01, 'train_loss': 2.425387382507324, 'epoch': 3.0}
